In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
from pyxlsb import convert_date
import numpy as np
import warnings
from pathlib import Path
warnings.simplefilter("ignore")

In [2]:
cwd = os.getcwd()
or_folder = str(Path(cwd).parents[0]) + "\\Today"
files = os.listdir(or_folder)
current_ops = [
    f
    for f in files
    if not (f.startswith(".")) and (f.endswith("xlsb") and not (f.startswith("#")))
]
ops_path = or_folder + "\\" + current_ops[0]
ops_data = pd.read_excel(ops_path, sheet_name="#Combined", engine="pyxlsb")
customer_data = pd.read_excel("Customer Complaint.xlsx")
esclated_order = pd.read_excel("Delayed Orders Posting.xlsx", sheet_name="ESCLATED ORDERS")
delayed_order = pd.read_excel("Delayed Orders Posting.xlsx", sheet_name="Delayed Orders")
order_export = pd.read_csv("orders_export_1.csv")

In [3]:
def convert_date_safe(date_str):
    if pd.isna(date_str):
        return np.nan
    return convert_date(date_str)

date_column = ["Delivery Date"]
for i in date_column:
 ops_data[i] = ops_data[i].apply(convert_date_safe)

In [4]:
category = "single"  # Change to "single" if you need only one day

today = datetime.today()
specific_date = today - timedelta(days=4)  # Choose a specific date (for single day)
start_date = today - timedelta(days=9)  # Start of range (for multiple days)
end_date = today - timedelta(days=4)  # End of range (for multiple days)

if category == "single":
    filtered_ops = ops_data[
        (ops_data['Delivery Date'] == specific_date.strftime('%Y-%m-%d')) &
        (ops_data['Reviews Check'] == True) &
        (ops_data['Country'] == 'IN')
    ]
elif category == "multiple":
    filtered_ops = ops_data[
        (ops_data['Delivery Date'].between(start_date.strftime('%Y-%m-%d'), end_date.strftime('%Y-%m-%d'))) &
        (ops_data['Reviews Check'] == True) &
        (ops_data['Country'] == 'IN')
    ]
else:
    print("Invalid category! Please use 'single' or 'multiple'.")
filtered_ops = filtered_ops[['Order Number']]

In [5]:
filtered_ops['Order Number'] = filtered_ops['Order Number'].astype(str).str.strip()
filtered_ops_unique = filtered_ops['Order Number'].unique()
filtered_ops_unique_df = pd.DataFrame(filtered_ops_unique, columns=['Order Number'])

customer_data['Customer Order Number'] = customer_data['Order Number'].astype(str).str.strip()
customer_data_unique = customer_data['Customer Order Number'].unique()
customer_data_unique_df = pd.DataFrame(customer_data_unique, columns=['Customer Order Number'])

esclated_order['escalated Order Number'] = esclated_order['ORDER NUMBER'].astype(str).str.strip()
esclated_order_unique = esclated_order['escalated Order Number'].unique()
escalated_order_unique_df = pd.DataFrame(esclated_order_unique, columns=['escalated Order Number'])

delayed_order['delayed Order Number'] = delayed_order['Order Number'].astype(str).str.strip()
delayed_order_unique = delayed_order['delayed Order Number'].unique()
delayed_order_unique_df = pd.DataFrame(delayed_order_unique, columns=['delayed Order Number'])

In [6]:
merged_df = pd.merge(filtered_ops_unique_df, customer_data_unique_df, 
                      left_on='Order Number', right_on='Customer Order Number', how='left')

merged_df = pd.merge(merged_df, escalated_order_unique_df, 
                      left_on='Order Number', right_on='escalated Order Number', how='left')

merged_df = pd.merge(merged_df, delayed_order_unique_df, 
                      left_on='Order Number', right_on='delayed Order Number', how='left')

In [7]:
filtered_merged_df = merged_df[
    (merged_df['Order Number'] != merged_df['Customer Order Number']) &
    (merged_df['Order Number'] != merged_df['escalated Order Number']) &
    (merged_df['Order Number'] != merged_df['delayed Order Number'])
]
filtered_merged_df["Order Number"] = filtered_merged_df["Order Number"].astype(int)

In [8]:
order_export["Name"] = order_export["Name"].str.replace("#", "")
order_export["Name"] = order_export["Name"].astype("int64")
order_export.drop_duplicates(subset=["Name"], keep="first", inplace=True)
order_export = order_export[["Name", "Shipping Name", "Shipping Phone"]]

In [9]:
review = filtered_merged_df.merge(
    order_export,
    how="left",
    left_on="Order Number",
    right_on="Name",
)

review["Name"] = review["Shipping Name"]
review["Phone Number"] = review["Shipping Phone"]
review["Country Code"] = 91
review.drop(columns=["Shipping Name", "Shipping Phone"], inplace=True)

review["Phone Number"] = review["Phone Number"].str.replace(r"[^0-9]", "", regex=True)
review["Phone Number"] = review["Phone Number"].str[-10:]

review.drop_duplicates(subset=["Phone Number"], inplace=True)

review = review[['Name', 'Order Number', 'Phone Number', 'Country Code']]

review.to_csv(str(today.strftime("%Y %m %d")) + " Review.csv", index=False)